In [3]:
# from ..utils.plot import plot_bbox_on_img, visualize_grid_on_img
from utils.convert_coordinates import yolo_to_voc
# from ..utils.convert_coordinates import voc_to_yolo, yolo_to_voc
import torch
from PIL import Image, ImageDraw
# from IPython.display import display
from torchvision import transforms



In [ ]:
def plot_bbox_on_img(img_path, bbox_coordinates):
    """
    Plots bboxes on a single image
    
        img : (tensor)
            Image tensor, this should be the resized image that we will pass to the model
        bbox_coordinates : (tensor)
             [class_id, X, Y, width, height]] in YOLO format, the coordinates have to be relative to the entire image, not a single cell.
    """
    # Convert from tensor to PIL image
    to_pil = transforms.ToPILImage()
    img = to_pil(img_tensor).convert("RGBA")
    # img = Image.open(img_path).convert("RGB")
    drawContext = ImageDraw.Draw(img)

    for box in bbox_coordinates:        
        class_id, x1, y1, x2, y2 = box
        drawContext.rectangle([x1, y1, x2, y2], outline='white', width=2)

        # Add (x, y) label
        label_text = f'{class_id}'
        
        drawContext.text((x1 + 5, y1 - 15), label_text, fill='white')  # position label above the top-left corner

    plot_x_y_axis(img, drawContext).show() # draw the x-y axes and plot

    # Display inline
    # display(img) # works for jupiter notebook only

In [ ]:
bboxes = torch.tensor([
    [0.0000, 0.1836, 0.3374, 0.0586, 0.1011],
    [1.0000, 0.4014, 0.3333, 0.0801, 0.1202],
    [0.0000, 0.6689, 0.3156, 0.0684, 0.1393]
])

In [ ]:
plot_bbox_on_img()